# Loading Sequences
Load in the sequences, store a dict of processed data.

In [1]:
from Bio import SeqIO
import pandas as pd
import numpy as np
# Add src directory from src
import sys
sys.path.append('../src/') 
import Core as C
# Import get_metadata file
#from get_metadata import get_sequence_metadata

In [3]:
import Bio.AlignIO.Interfaces
from Bio import pairwise2
import joblib
import multiprocessing  # Check num of cores
import pickle


In [4]:
%load_ext autoreload

In [5]:
autoreload

In [6]:
data_file = "../data/FASTA_samples_no_identical_seqs.fa"
df = C.get_sequence_metadata(data_file, save=False)

In [7]:
df.head(10)

,accession_id,strain,year,month,day,segname,segnum,length
0,X17221,A/CHR/157/83,1983,,,4 (HA),{segment number},1752
1,X05907,A/England/321/1977,1977,,,4 (HA),{segment number},1762
2,V01085,A/Aichi/2/1968,1968,,,4 (HA),{segment number},1765
3,V01086,A/Victoria/3/1975,1975,,,4 (HA),{segment number},1768
4,V01098,A/Victoria/3/1975,1975,,,4 (HA),{segment number},1768
5,V01103,A/NT/60/68/29c,1968,,,4 (HA),{segment number},1765
6,X15281,A/Chile/1/1983,1983,,,6 (NA),{segment number},1461
7,V01088,A/Puerto Rico/8/1934,1934,,,4 (HA),{segment number},1778
8,D13580,A/Izumi/5/65(R),1965,,,4 (HA),{segment number},1773
9,D13579,A/Izumi/5/65,1965,,,4 (HA),{segment number},1773


In [8]:
data = C.query_metadata(df, 1990,strain="Wash", seq_name ="HA").head(10)

In [9]:
autoreload

In [10]:
data_NA_Nstates = C.query_metadata(df, 1990, strain="", seq_name ="NA", strain_except="Wash")
data_NA_Nstates

,accession_id,strain,year,month,day,segname,segnum,length
51,D31945,A/Hokkaido/2/1992,1992,,,6 (NA),{segment number},1461
62,U53166,A/Wisconsin/4754/1994,1994,,,6 (NA),{segment number},1458
70,U42636,A/Shanghai/24/90,1990,,,6 (NA),{segment number},1410
72,U42770,A/Beijing/32/92,1992,,,6 (NA),{segment number},1410
73,U42771,A/Beijing/46/92,1992,,,6 (NA),{segment number},1410
...,...,...,...,...,...,...,...,...
62919,MT499611,A/California/17/2020,2020,01,23,6 (NA),{segment number},1433
62922,MT499624,A/California/51/2020,2020,02,18,6 (NA),{segment number},1433
62925,MT499640,A/California/54/2020,2020,03,16,6 (NA),{segment number},1433
62928,MT499656,A/Hawaii/25/2020,2020,03,13,6 (NA),{segment number},1433


In [11]:
data_HA_Nstates = C.query_metadata(df, 1990, strain="", seq_name ="HA", strain_except="Wash")
data_HA_Nstates

,accession_id,strain,year,month,day,segname,segnum,length
10,D13574,A/Suita/1/1989/(R),2000,,,4 (HA),{segment number},1778
48,L24362,A/Maryland/12/1991,1991,,,4 (HA),{segment number},1738
57,U26830,A/Beijing/32/1992,1992,,,4 (HA),{segment number},1701
61,U53162,A/Wisconsin/4754/1994,1994,,,4 (HA),{segment number},1778
101,AF028709,A/Hong Kong/156/97,1997,,,4 (HA),{segment number},1741
...,...,...,...,...,...,...,...,...
62929,MT499662,A/Montana/16/2020,2020,03,16,4 (HA),{segment number},1752
62930,MT499670,A/Oregon/15/2020,2020,03,19,4 (HA),{segment number},1752
62931,MT499678,A/Oregon/16/2020,2020,03,21,4 (HA),{segment number},1752
62932,MT499686,A/Utah/18/2020,2020,03,16,4 (HA),{segment number},1752


In [12]:
data_HA_Wstates = C.query_metadata(df, 1990, seq_name ="HA", strain="Wash")
data_HA_Wstates

,accession_id,strain,year,month,day,segname,segnum,length
4045,EU100698,A/Washington/01/2007,2007,01,10,4 (HA),{segment number},1701
4612,EU199367,A/Washington/14/2007,2007,01,26,4 (HA),{segment number},1701
4615,EU199370,A/Washington/20/2007,2007,03,04,4 (HA),{segment number},1701
4720,CY026603,A/Washington/UR06-0225/2007,2007,02,08,4 (HA),{segment number},1721
4756,CY026771,A/Washington/UR06-0252/2007,2007,02,12,4 (HA),{segment number},1724
...,...,...,...,...,...,...,...,...
62754,MT342661,A/Washington/17/2020,2020,01,29,4 (HA),{segment number},1737
62756,MT342674,A/Washington/17/2020,2020,01,29,4 (HA),{segment number},1737
62825,MT422953,A/Washington/20/2020,2020,02,19,4 (HA),{segment number},1737
62934,MT499702,A/Washington/25/2020,2020,02,26,4 (HA),{segment number},1752


In [13]:
data_NA_Wstates = C.query_metadata(df, 1990,seq_name ="NA", strain="Wash")
data_NA_Wstates

,accession_id,strain,year,month,day,segname,segnum,length
71,U42637,A/Washington/15/91,1991,,,6 (NA),{segment number},1410
86,U43420,A/Washington/26/93,1993,,,6 (NA),{segment number},1410
3977,EU100510,A/Washington/6/2001,2001,10,29,6 (NA),{segment number},1413
4011,EU100549,A/Washington/v13/2003,2003,05,06,6 (NA),{segment number},1418
4031,EU100574,A/Washington/27/2003,2003,12,06,6 (NA),{segment number},1418
...,...,...,...,...,...,...,...,...
62682,MT331507,A/Washington/18/2020,2020,02,10,6 (NA),{segment number},1433
62755,MT342663,A/Washington/17/2020,2020,01,29,6 (NA),{segment number},1441
62809,MT419706,A/Washington/21/2020,2020,02,23,6 (NA),{segment number},1433
62810,MT419715,A/Washington/23/2020,2020,02,28,6 (NA),{segment number},1433


In [14]:
autoreload

For some hardware with good single core performance, this is going to be faster, but generally not.
We see a 50% speedup with multicore (at least on my machine), and that only seems to be using 4 threads.

In [57]:
# %%time
parser = C.parser.Parser()
parser.build_metadata_from_df(data_HA_Wstates)
parser.build_metadata_from_df(data_NA_Wstates)
parser.build_data(data_file)
parser.parse_ref_seqs("../data/Reference_seqs_cds_Flu_NCBI_Chr_4_6.fa")
#parsed_data = parser.parse_data(mc=False)

Setting Metadata attribute


For some hardware, this block with `loky` backend parallelism is going to be faster.

In [15]:
%%time
parser = C.parser.Parser()
parser.build_metadata_from_df(data_HA_Wstates)
parser.build_metadata_from_df(data_NA_Wstates)
parser.build_data(data_file)
parser.parse_ref_seqs("../data/Reference_seqs_cds_Flu_NCBI_Chr_4_6.fa")
test_data = parser.parse_data(mc=True)

file_handle_str = './Pickle_storage/test_data_WA.pkl'
with open(file_handle_str, 'wb') as file_handle:
    pickle.dump(test_data, file_handle)

Setting Metadata attribute


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  88 out of 933 | elapsed:    6.6s remaining:  1.1min
[Parallel(n_jobs=16)]: Done 182 out of 933 | elapsed:   11.9s remaining:   49.2s
[Parallel(n_jobs=16)]: Done 276 out of 933 | elapsed:   17.4s remaining:   41.4s
[Parallel(n_jobs=16)]: Done 370 out of 933 | elapsed:   22.6s remaining:   34.4s
[Parallel(n_jobs=16)]: Done 464 out of 933 | elapsed:   27.9s remaining:   28.2s
[Parallel(n_jobs=16)]: Done 558 out of 933 | elapsed:   34.2s remaining:   23.0s
[Parallel(n_jobs=16)]: Done 652 out of 933 | elapsed:   42.4s remaining:   18.3s
[Parallel(n_jobs=16)]: Done 746 out of 933 | elapsed:   50.6s remaining:   12.7s
[Parallel(n_jobs=16)]: Done 840 out of 933 | elapsed:   58.7s remaining:    6.5s


CPU times: user 3.55 s, sys: 496 ms, total: 4.05 s
Wall time: 1min 7s


[Parallel(n_jobs=16)]: Done 933 out of 933 | elapsed:  1.1min finished


In [59]:
%%time
parser = C.parser.Parser()
parser.build_metadata_from_df(data_HA_Wstates)
parser.build_metadata_from_df(data_NA_Wstates)
parser.build_data(data_file)
parser.parse_ref_seqs("../data/Reference_seqs_cds_Flu_NCBI_Chr_4_6.fa")
test_data = parser.parse_data(mc=True)

file_handle_str = './Pickle_storage/test_data_WA.pkl'
with open(file_handle_str, 'wb') as file_handle:
    pickle.dump(test_data, file_handle)

Setting Metadata attribute


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  88 out of 933 | elapsed:    7.7s remaining:  1.2min
[Parallel(n_jobs=16)]: Done 182 out of 933 | elapsed:   13.7s remaining:   56.6s
[Parallel(n_jobs=16)]: Done 276 out of 933 | elapsed:   19.8s remaining:   47.2s
[Parallel(n_jobs=16)]: Done 370 out of 933 | elapsed:   25.7s remaining:   39.1s
[Parallel(n_jobs=16)]: Done 464 out of 933 | elapsed:   31.6s remaining:   31.9s
[Parallel(n_jobs=16)]: Done 558 out of 933 | elapsed:   38.5s remaining:   25.9s
[Parallel(n_jobs=16)]: Done 652 out of 933 | elapsed:   47.9s remaining:   20.6s
[Parallel(n_jobs=16)]: Done 746 out of 933 | elapsed:   56.7s remaining:   14.2s
[Parallel(n_jobs=16)]: Done 840 out of 933 | elapsed:  1.1min remaining:    7.2s


CPU times: user 3.67 s, sys: 504 ms, total: 4.18 s
Wall time: 1min 14s


[Parallel(n_jobs=16)]: Done 933 out of 933 | elapsed:  1.2min finished


In [ ]:
%%time
parser = C.parser.Parser()
parser.build_metadata_from_df(data_HA_Nstates)
parser.build_metadata_from_df(data_NA_Nstates)
parser.build_data(data_file)
parser.parse_ref_seqs("../data/Reference_seqs_cds_Flu_NCBI_Chr_4_6.fa")
train_data = parser.parse_data(mc=True)
file_handle_str = './Pickle_storage/train_data.pkl'
with open(file_handle_str, 'wb') as file_handle:
    pickle.dump(train_data, file_handle)

Setting Metadata attribute


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done 6092 out of 60964 | elapsed: 14.9min remaining: 133.8min
[Parallel(n_jobs=16)]: Done 12189 out of 60964 | elapsed: 29.9min remaining: 119.5min
[Parallel(n_jobs=16)]: Done 18286 out of 60964 | elapsed: 44.9min remaining: 104.8min
[Parallel(n_jobs=16)]: Done 24383 out of 60964 | elapsed: 60.0min remaining: 90.0min
[Parallel(n_jobs=16)]: Done 30480 out of 60964 | elapsed: 75.0min remaining: 75.0min
[Parallel(n_jobs=16)]: Done 36577 out of 60964 | elapsed: 90.4min remaining: 60.2min
[Parallel(n_jobs=16)]: Done 42674 out of 60964 | elapsed: 106.2min remaining: 45.5min
[Parallel(n_jobs=16)]: Done 48771 out of 60964 | elapsed: 122.0min remaining: 30.5min
[Parallel(n_jobs=16)]: Done 54868 out of 60964 | elapsed: 137.8min remaining: 15.3min


In [219]:
import pickle
file_handle_str = './Pickle_storage/parsed_dict_data.pkl'
with open(file_handle_str, 'wb') as file_handle:
    pickle.dump(parsed_data, file_handle)

In [ ]:
with open(file_handle_str, 'rb') as file_handle


In [172]:
n_cores = multiprocessing.cpu_count()
print(n_cores)

32


In [175]:
%%time
parser = C.parser.Parser()
parser.build_metadata_from_df(data)
parser.build_data(data_file)
parser.parse_ref_seqs("../data/Reference_seqs_cds_Flu_NCBI_Chr_4_6.fa")
parsed_data = parser.parse_data(mc=True)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 105 out of 1000 | elapsed:  1.1min remaining:  9.4min
[Parallel(n_jobs=4)]: Done 217 out of 1000 | elapsed:  1.6min remaining:  5.9min
[Parallel(n_jobs=4)]: Done 329 out of 1000 | elapsed:  1.6min remaining:  3.4min
[Parallel(n_jobs=4)]: Done 441 out of 1000 | elapsed:  1.6min remaining:  2.1min
[Parallel(n_jobs=4)]: Done 553 out of 1000 | elapsed:  1.6min remaining:  1.3min
[Parallel(n_jobs=4)]: Done 665 out of 1000 | elapsed:  1.6min remaining:   49.7s
[Parallel(n_jobs=4)]: Done 777 out of 1000 | elapsed:  1.6min remaining:   28.3s
[Parallel(n_jobs=4)]: Done 889 out of 1000 | elapsed:  1.6min remaining:   12.3s


KeyboardInterrupt: 

In [141]:
parsed_data['scores']

[1174.0,
 1173.0,
 1173.0,
 1176.0,
 1177.0,
 1360.0,
 1361.0,
 1175.0,
 1359.0,
 1359.0,
 1361.0,
 1360.0,
 1366.0,
 1173.0,
 1170.0,
 1175.0,
 1359.0,
 1361.0,
 1177.0,
 1366.0,
 1362.0,
 1359.0,
 1695.0,
 1694.0,
 1361.0,
 1173.0,
 1172.0,
 1693.0,
 1694.0,
 1694.0,
 1174.0,
 1175.0,
 1363.0,
 1690.0,
 1173.0,
 1174.0,
 1689.0,
 1688.0,
 1691.0,
 1688.0,
 1690.0,
 1689.0,
 1692.0,
 1675.0,
 1688.0,
 1694.0,
 1689.0,
 1693.0,
 1694.0,
 1687.0,
 1690.0,
 1686.0,
 1688.0,
 1683.0,
 1680.0,
 1686.0,
 1684.0,
 1686.0,
 1176.0,
 1168.0,
 1174.0,
 1175.0,
 1178.0,
 1677.0,
 1677.0,
 1676.0,
 1674.0,
 1676.0,
 1675.0,
 1673.0,
 1171.0,
 1170.0,
 1172.0,
 1174.0,
 1171.0,
 1171.0,
 1175.0,
 1174.0,
 1172.0,
 1173.0,
 1171.0,
 1170.0,
 1168.0,
 1170.0,
 1171.0,
 1171.0,
 1171.0,
 1171.0,
 1674.0,
 1168.0,
 1168.0,
 1171.0,
 1169.0,
 1169.0,
 1672.0,
 1669.0,
 1670.0,
 1668.0,
 1672.0,
 1671.0]

In [ ]:
print(np.asarray(parsed_data['labels'][0], dtype=int))

In [79]:
parser.metadata['segname']

{'D13574': '4 (HA)',
 'L24362': '4 (HA)',
 'U26830': '4 (HA)',
 'U53162': '4 (HA)',
 'AF028709': '4 (HA)',
 'AF028020': '4 (HA)',
 'AF046088': '4 (HA)',
 'AF046096': '4 (HA)',
 'AF046097': '4 (HA)',
 'AF046098': '4 (HA)',
 'AB019354': '4 (HA)',
 'AB019355': '4 (HA)',
 'AB019356': '4 (HA)',
 'AB019357': '4 (HA)',
 'AJ404626': '4 (HA)',
 'AJ404627': '4 (HA)',
 'AF363502': '4 (HA)',
 'AF363503': '4 (HA)',
 'AF363504': '4 (HA)',
 'AY032978': '4 (HA)',
 'AF017270': '4 (HA)',
 'AY035588': '4 (HA)',
 'AY035589': '4 (HA)',
 'AY035590': '4 (HA)',
 'AY035591': '4 (HA)',
 'AF382318': '4 (HA)',
 'AF382319': '4 (HA)',
 'AF382321': '4 (HA)',
 'AF382323': '4 (HA)',
 'AF386773': '4 (HA)',
 'AF386774': '4 (HA)',
 'AF386775': '4 (HA)',
 'AF386776': '4 (HA)',
 'AF386777': '4 (HA)',
 'AF386778': '4 (HA)',
 'AF386780': '4 (HA)',
 'AF386781': '4 (HA)',
 'AF017272': '4 (HA)',
 'AY271794': '4 (HA)',
 'AY282756': '4 (HA)',
 'AY282757': '4 (HA)',
 'AY289927': '4 (HA)',
 'AY289928': '4 (HA)',
 'AY289929': '4 (HA

In [91]:
parser.refs

{'NC_026433.1': SeqRecord(seq=Seq('ATGAAGGCAATACTAGTAGTTCTGCTATATACATTTGCAACCGCAAATGCAGAC...TAA'), id='NC_026433.1', name='NC_026433.1', description='NC_026433.1 Influenza A virus (A/California/07/2009(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds', dbxrefs=[]),
 'NC_026434.1': SeqRecord(seq=Seq('ATGAATCCAAACCAAAAGATAATAACCATTGGTTCGGTCTGTATGACAATTGGA...TAA'), id='NC_026434.1', name='NC_026434.1', description='NC_026434.1 Influenza A virus (A/California/07/2009(H1N1)) segment 6 neuraminidase (NA) gene, complete cds', dbxrefs=[]),
 'NC_004908.1:32-1714': SeqRecord(seq=Seq('ATGGAAACAATATCACTAATAACTATACTACTAGTAGTAACAGCAAGCAATGCA...TAA'), id='NC_004908.1:32-1714', name='NC_004908.1:32-1714', description='NC_004908.1:32-1714 Influenza A virus ha gene for Hemagglutinin, genomic RNA, strain A/Hong Kong/1073/99(H9N2)', dbxrefs=[]),
 'NC_004909.1:1-1404': SeqRecord(seq=Seq('ATGAATCCAAATCAAAAGATAATAGCACTTGGCTCTGTTTCTATAACTATTGCG...TAA'), id='NC_004909.1:1-1404', name='NC_004909.1:1-1404'